In [13]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import pandas as pd
import datetime

In [16]:
#rushing/passing/receiving/interceptions/sacks(e.1982)/tackles(e.2001)/Field_Goals/Punting(e.1939)

now = datetime.datetime.now()

statisticCategory = input("Which statistical category (rushing/passing/receiving/interceptions/sacks/tackles/Field_Goals/Punting:")
if (statisticCategory.upper() != "SACKS") & (statisticCategory.upper() != "RUSHING") & (statisticCategory.upper() != "PASSING") & (statisticCategory.upper() != "RECEIVING") & (statisticCategory.upper() != "INTERCEPTIONS") & (statisticCategory.upper() != "TACKLES") & (statisticCategory.upper() != "FIELD_GOALS") & (statisticCategory.upper() != "PUNTING"):
    raise Exception("ERROR: statistical category must be one of the listed")


season_type = input("What type of season (pre, reg, post): ")
if (season_type.upper() != 'PRE') & (season_type.upper() != 'REG') & (season_type.upper() != "POST") :
    raise Exception("ERROR: Season Type needs to be either pre, reg, or post")
        
start_season = input("Starting Year: ")
if (statisticCategory.upper() == "SACKS") & (int(start_season) < 1982):
    raise Exception("ERROR: For sacks, the starting year can't be less than 1932")
if (statisticCategory.upper() == "TACKLES") & (int(start_season) < 2001):
    raise Exception("ERROR: For tackles, the starting year can't be less than 2001")
if (statisticCategory.upper() == "PUNTING") & (int(start_season) < 1939):
    raise Exception("ERROR: For punting, the starting year can't be less than 1939")
if (int(start_season) < 1932) | int(start_season) > now.year :
    raise Exception("ERROR: The starting year can't be less than 1932")
    
last_season = input("Last Year: ")
if (int(last_season) < 1932) | int(last_season) > now.year:
    raise Exception("ERROR: The last year can't be less than 1932 and more than the current year")

Which statistical category (rushing/passing/receiving/interceptions/sacks/tackles/Field_Goals/Punting:rushing
What type of season (pre, reg, post): reg
Starting Year: 2019
Last Year: 2019


In [17]:
for year in range(int(start_season), int(last_season) + 1):
    print(year)

2019


In [ ]:
x = []
for season in range(int(start_season), int(last_season) + 1):
    p = 1
    payload = {"statisticCategory":statisticCategory.upper(),"seasonType":season_type.upper(),"d-447263-p":str(p),"season":season}
    url = 'http://www.nfl.com/stats/categorystats'
    response = requests.get(url,params=payload)
    time.sleep(1)
    print("Response:", response.status_code,response.url) #200 means it went through
    soup = BeautifulSoup(response.text,'html.parser')
    
    #The website table is partitioned into multiple pages.  We need to know how many pages to iterate

    pagingText = soup.find("span", {"class": "linkNavigation floatRight"})
    print(pagingText == None)
    if(pagingText == None):
        pagingLinks = 0
        pages = 0
        print("Number of page links:",0)
        columnHeader = []
        header = soup.find('table', attrs={'id':'result'}).find_all('th')
        for c in header:
            columnHeader.append(c.text)
        columnHeader = [c.strip('\n') for c in columnHeader]
        if (statisticCategory.upper() == "TACKLES") | (statisticCategory.upper() == "SACKS") | (statisticCategory.upper() == "INTERCEPTIONS"):
            columnHeader = columnHeader[4:]
        print(columnHeader)

        tableRows = []

        for pg in range(1,2):

            #pause our code for a second so that we are not spamming the website with requests. This helps us avoid getting flagged as a spammer.
            time.sleep(5)

            #update page number in payload
            payload["d-447263-p"] = pg
            response = requests.get(url,params=payload)
            print("Page:",pg,"url:",response.url)
            soup = BeautifulSoup(response.text,'html.parser')
            table = soup.find('table', attrs={'id':'result'})
            rows = table.find_all('tr')

            for row in rows:
                cols = row.find_all('td')
                if cols:
                    cols = [ele.text.strip() for ele in cols]
                    tableRows.append([ele for ele in cols if ele]) # Get rid of empty values

    else:
        pagingLinks = pagingText.findAll("a")
        pages = len(pagingLinks)
        print("Number of page links:",len(pagingLinks))
        columnHeader = []
        header = soup.find('table', attrs={'id':'result'}).find_all('th')
        for c in header:
            columnHeader.append(c.text)
        columnHeader = [c.strip('\n') for c in columnHeader]
        if (statisticCategory.upper() == "TACKLES") | (statisticCategory.upper() == "SACKS") | (statisticCategory.upper() == "INTERCEPTIONS"):
            columnHeader = columnHeader[4:]
        print(columnHeader)

        tableRows = []

        for pg in range(1,pages+1):

            #pause our code for a second so that we are not spamming the website with requests. This helps us avoid getting flagged as a spammer.
            time.sleep(5)

            #update page number in payload
            payload["d-447263-p"] = pg
            response = requests.get(url,params=payload)
            print("Page:",pg,"url:",response.url)
            soup = BeautifulSoup(response.text,'html.parser')
            table = soup.find('table', attrs={'id':'result'})
            rows = table.find_all('tr')

            for row in rows:
                cols = row.find_all('td')
                if cols:
                    cols = [ele.text.strip() for ele in cols]
                    tableRows.append([ele for ele in cols if ele]) # Get rid of empty values

    resultsDF = pd.DataFrame(tableRows, columns=columnHeader)
    resultsDF['Season'] = season
    display(resultsDF.head(1),resultsDF.shape)
    x.append(resultsDF)

In [ ]:
x[0]
x[0].append(x[1:len(x)+1])

In [ ]:
results = x[0].append(x[1:len(x)+1])

In [ ]:
results

In [ ]:
baseFileName = "NFL_Stats_Demo_"

results.to_excel(baseFileName + statisticCategory + "_" + season_type + "_" + start_season + "_" + last_season + ".xlsx",index=False)
results.to_csv(baseFileName + statisticCategory + "_" + season_type + "_" + start_season + "_" + last_season + ".csv",index=False)